In [ ]:
# グラフを移動拡大縮小したい場合
# %matplotlib notebook
# グラフをインラインで表示
%matplotlib inline

#Cell_1.
import gc
import math
import pickle

import warnings
warnings.simplefilter('ignore')

import mylibs.mylib1 as mylib1
import mylibs.mylib2 as mylib2
import mylibs.mylib3 as mylib3

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

# 表示する行・列の数を設定
pd.options.display.max_rows = 8
pd.options.display.max_columns = 12

# matplotlib のフォントを設定
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'

In [ ]:
#Cell_2.
df_bp = pd.read_csv('public_stats/original/nhns_bp_mid_r2.csv', skiprows=0)

df_bp.head()

In [ ]:
#Cell_3.
df_bp = df_bp.loc[:, df_bp.columns[df_bp.iloc[0] != '%']]

df_bp.head()

In [ ]:
#Cell_4.
df_bp = df_bp.loc[1:, df_bp.columns[:10]]
df_bp = df_bp.fillna(method='ffill')

df_bp.head()

In [ ]:
#Cell_5.
df_bp.columns = ['sex', 'bp_type', 'bp_value', 'total',
                 '20s', '30s', '40s', '50s', '60s', '70s']

df_bp.tail()

In [ ]:
#Cell_6.
df_bp = df_bp.iloc[:50]
df_bp = df_bp.loc[df_bp.bp_value != '総数', :]

display(df_bp.head())

In [ ]:
#Cell_7.
df_bp.sex[df_bp.sex == '男性'] = 'M'
df_bp.sex[df_bp.sex == '女性'] = 'F'
df_bp.bp_type[df_bp.bp_type == '収縮期（最高）血圧'] = 'sbp'
df_bp.bp_type[df_bp.bp_type == '拡張期（最低）血圧'] = 'dbp'
df_bp = df_bp.reset_index()
del df_bp['index']

df_bp

In [ ]:
#Cell_8.
#visualization.
import seaborn as sns
df_m_sbp=df_bp[(df_bp['sex']=='M') & (df_bp['bp_type']=='sbp')]
df_m_dbp=df_bp[(df_bp['sex']=='M') & (df_bp['bp_type']=='dbp')]
a=df_bp[(df_bp['sex']=='F') & (df_bp['bp_type']=='dbp')]

df_m_sbp['bp_value_class'] = [85,95,105,115,125,135,145,155,165,175,185]
df_m_dbp['bp_value_class'] = [35,45,55,65,75,85,95,105,115,125,135,145]
#display(df_m_sbp.head())
display(df_m_dbp.head())
display(df_m_sbp.tail())


In [ ]:
a.head(), a.tail()

In [ ]:
#Cell_9.
print(df_m_sbp.info())
print(df_m_dbp.shape)

In [ ]:
#Cell_10.
df_m_sbp['total']=df_m_sbp['total'].astype("int")
df_m_dbp['total']=df_m_dbp['total'].astype("int")
df_m_sbp.head(2)

In [ ]:
%matplotlib inline
#Cell_11.
df_m_sbp.plot.bar(x='bp_value_class',y='total')
df_m_dbp.plot.bar(x='bp_value_class',y='total')

In [ ]:
#Cell_12.
df_bp.bp_value.unique()

In [ ]:
#Cell_13.
sbp_min = np.concatenate([[70], np.arange(9, 19) * 10])

sbp_min

In [ ]:
#Cell_14.
sbp_max = np.concatenate([sbp_min[1:] - 1, [199]])

sbp_max

In [ ]:
#Cell_15.
dbp_min = np.arange(4, 15) * 10

dbp_min

In [ ]:
#Cell_16.
dbp_max = np.concatenate([dbp_min[1:] - 1, [159]])

dbp_max

In [ ]:
#Cell_17.
dict_sbp_max = {k: v for (k, v) in zip(sbp_min, sbp_max)}
dict_dbp_max = {k: v for (k, v) in zip(dbp_min, dbp_max)}

dict_sbp_max

In [ ]:
#Cell_18.
df_bp = df_bp[df_bp['bp_value'] != '40mmHg未満'].copy()
df_bp['bp_min'] = np.tile(np.concatenate([sbp_min, dbp_min]), 2)
del df_bp['bp_value']
del df_bp['total']

df_bp

In [ ]:
#Cell_19.
df_bp.dtypes

In [ ]:
#Cell_20.
df_bp[df_bp.columns[2:]] = df_bp[df_bp.columns[2:]].copy().astype('int16')

df_bp.dtypes

In [ ]:
#Cell_21.
ages = df_bp.columns[2:8]  # ages = ['20s', ... , '70s']
# SBP の男性回答者数
df_bp.loc[(df_bp.sex == 'F') & (df_bp.bp_type == 'sbp'), ages].sum()

In [ ]:
#Cell_22.
# DBP の男性回答者数
df_bp.loc[(df_bp.sex == 'F') & (df_bp.bp_type == 'dbp'), ages].sum()

In [ ]:
#Cell_23.
sex = 'M'
bp_type = 'sbp'
age = '20s'

# 指定する (sex, bp_type, age) における回答者数リスト
ix = (df_bp.sex == sex) & (df_bp.bp_type == bp_type)
n = df_bp.loc[ix, age].values

# リスト n に対応する bp_min リスト
bp_min = df_bp.loc[ix, 'bp_min'].values

print('n =', n)
print('bp_min =', bp_min)

In [ ]:
#Cell_24.
bp_min.repeat(n)

In [ ]:
#Cell_25.
bp_min.repeat(n)
dict_bp_min_dist = {}
dict_bp_min_dist['M'] = {}
dict_bp_min_dist['M']['dbp'] = {}
dict_bp_min_dist['M']['sbp'] = {}
dict_bp_min_dist['F'] = {}
dict_bp_min_dist['F']['dbp'] = {}
dict_bp_min_dist['F']['sbp'] = {}

for sex in ['M', 'F']:
    for bp_type in ['sbp', 'dbp']:
        for age in ages:
            dict_bp_min_dist[sex][bp_type][age] = {}
            ix = (df_bp.sex == sex) & (df_bp.bp_type == bp_type)
            n = df_bp.loc[ix, age].values
            bp_min = df_bp.loc[ix, 'bp_min'].values
            dict_bp_min_dist[sex][bp_type][age] = bp_min.repeat(n)

In [ ]:
#Cell_26.
dict_bp_min_dist['M']['sbp']['20s']

In [ ]:
#Cell_27.
import pickle

f = open('./public_stats/processed/dict_bp_min_dist.binaryfile', 'wb')  # wb: Write Binary
pickle.dump(dict_bp_min_dist, f)
f.close()

In [ ]:
#Cell_28.
f = open('./public_stats/processed/dict_bp_min_dist.binaryfile', 'rb')  # rb: Read Binary
dict_bp_min_dist = pickle.load(f)
f.close()

dict_bp_min_dist['M']['sbp']['20s']

In [ ]:
#Cell_29.
f = open('./public_stats/processed/dict_dbp_max.binaryfile', 'wb')  # wb: Write Binary
pickle.dump(dict_dbp_max, f)
f.close()

f = open('./public_stats/processed/dict_sbp_max.binaryfile', 'wb')  # wb: Write Binary
pickle.dump(dict_sbp_max, f)
f.close()

In [ ]:
#Cell_30.
f = open('./public_stats/processed/dict_dbp_max.binaryfile', 'rb')  # rb: Read Binary
dict_dbp_min = pickle.load(f)
f.close()

dict_dbp_min

In [ ]:
#Cell_31.
f = open('./public_stats/processed/dict_sbp_max.binaryfile', 'rb')  # rb: Read Binary
dict_sbp_min = pickle.load(f)
f.close()

dict_sbp_min

In [ ]:
#Cell_32.
df_bp.to_csv('./public_stats/processed/df_bp.csv', index=False)

df_bp = pd.read_csv('./public_stats/processed/df_bp.csv')
df_bp

### 血圧情報の擬似生成と可視化

In [ ]:
#Cell_33.
def calc_bp_class(dbp, sbp):
    return ((dbp >= 80) | (sbp >= 120)) * 1 \
         + ((dbp >= 85) | (sbp >= 130)) * 1 \
         + ((dbp >= 90) | (sbp >= 140)) * 1 \
         + ((dbp >= 100) | (sbp >= 160)) * 1 \
         + ((dbp >= 110) | (sbp >= 180)) * 1


# 年齢ごとにマーカーの色を変える
dict_color = {'20s': 'r', '30s': 'orange', '40s': 'y',
              '50s': 'g', '60s': 'b', '70s': 'purple'}

# マーカーを血圧分類値にする
dict_marker = {0: '$0$', 1: '$1$', 2: '$2$', 3: '$3$', 4: '$4$', 5: '$5$'}

In [ ]:
#Cell_34.
# 表示の大きさと解像度を設定
fig = plt.figure(figsize=(8, 12))
fig.suptitle('BP Class Distribution')

# 6つの散布図を同時に表示
ax_My = fig.add_subplot(321)  # Male   Young
ax_Fy = fig.add_subplot(322)  # Female Young
ax_Mm = fig.add_subplot(323)  # Male   Middle
ax_Fm = fig.add_subplot(324)  # Female Middle
ax_Me = fig.add_subplot(325)  # Male   Elderly
ax_Fe = fig.add_subplot(326)  # Female Elderly

rs = 0
np.random.seed(rs)

sexes = ['M', 'F']
ages = df_bp.columns[2:8]

########以下を埋めで実装せよ。##########
for sex in sexes:
    for age in ages:
        sbp_min = dict_bp_min_dist[sex]['sbp'][age]
        sbp_max = [dict_sbp_max[sbpm] for sbpm in sbp_min]
        dbp_min = dict_bp_min_dist[sex]['dbp'][age]
        dbp_max = [dict_dbp_max[dbpm] for dbpm in dbp_min]
        n = len(sbp_min)

        # SBP を乱数生成
        sbp = np.random.rand(n) * (sbp_max - sbp_min) + sbp_min
        rs = rs + 1
        np.random.seed(rs)

        # DBP を乱数生成
        dbp = np.random.rand(n) * (dbp_max - dbp_min) + dbp_min
        rs = rs + 1
        np.random.seed(rs)

        # 軸を選ぶ
        if sex == 'M' and (age == '20s' or age == '30s'):
            ax = ax_My
        elif sex == 'M' and (age == '40s' or age == '50s'):
            ax = ax_Mm
        elif sex == 'M' and (age == '60s' or age == '70s'):
            ax = ax_Me
        elif sex == 'F' and (age == '20s' or age == '30s'):
            ax = ax_Fy
        elif sex == 'F' and (age == '40s' or age == '50s'):
            ax = ax_Fm
        else:
            ax = ax_Fe

        # 各性別・年齢区分における最初の人だけ凡例をつける
        marker = calc_bp_class(dbp, sbp)
        print(marker)
        for m in np.arange(6):
            x = dbp[marker == m]
            y = sbp[marker == m]
            if m == 0:
                ax.scatter(x, y, c=dict_color[age], marker=dict_marker[m],
                           s=25, linewidth=0.1, label='BP class of Age ' + age)
            else:
                ax.scatter(x, y, c=dict_color[age], marker=dict_marker[m],
                           s=25, linewidth=0.1)


axes = [ax_My, ax_Mm, ax_Me, ax_Fy, ax_Fm, ax_Fe]

# 血圧分類の境界を作成
for ax in axes:
    # vertical lines
    ax.plot([80, 80], [70, 120], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([85, 85], [70, 130], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([90, 90], [70, 140], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([100, 100], [70, 160], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([110, 110], [70, 180], linestyle='--', c='grey', linewidth=0.2)
    # horizontal lines
    ax.plot([40, 80], [120, 120], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([40, 85], [130, 130], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([40, 90], [140, 140], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([40, 100], [160, 160], linestyle='--', c='grey', linewidth=0.2)
    ax.plot([40, 110], [180, 180], linestyle='--', c='grey', linewidth=0.2)

# 各散布図にタイトルを追加
titles = ['Young Male (age:20-39)', 'Middle Male (age:40-59)',
          'Elderly Male (age:60-)', 'Young Female (age:20-39)',
          'Middle Female (age40-59)', 'Elderly Female (age:60-)']
for ax, title in zip(axes, titles):
    ax.set_xlabel('DBP', fontsize=10)
    ax.set_ylabel('SBP', fontsize=10)
    ax.set_xlim(40, 140)
    ax.set_ylim(70, 200)
    ax.set_title(title, fontsize=10)
    ax.tick_params(labelsize=8)
    ax.legend(fontsize=10)

# 散布図同士の間隔調整
plt.tight_layout()
plt.subplots_adjust(top=0.93)